In [ ]:
import requests
from bs4 import BeautifulSoup
import datetime as DT
import pymysql

In [ ]:
companies=[] #Список, куда собираются все объектв компаний

In [ ]:
#Для хранения данных, собранных с сайта, создал клас чоответсвующими полями
class Company():
    id_=None #id сразу беру с сайта, чтобы поместить объект в БД
    name=None
    ogrn=None
    okpo=None
    status=None
    regDate=None
    capital=None
    
    def __init__(self, id_):
        self.id_=int(id_.replace('/id/',''))
    
    def setCapital(self, capital):
        self.capital=int(capital.replace(' руб.','').replace(' ',''))
        
    def setDate(self, regDate):
        self.regDate=DT.datetime.strptime(regDate, '%d.%m.%Y').date()
    
    def printInfo(self):
        print(self.id_, self.name, self.ogrn, self.status, self.regDate, self.capital)

In [ ]:
#Основной метод
def mainParser(id_):
    r=requests.get(mainUrl+id_)
    compInfo=BeautifulSoup(r.content,'lxml')
    c=Company(id_)
    
    c.name=compInfo.find('div',class_='company-name').text
    c.ogrn=compInfo.find('span',{'id': 'clip_ogrn'}).text
    c.okpo=compInfo.find('span',{'id': 'clip_okpo'}).text
    c.status=compInfo.find('div',class_='company-status').text
    
    for ci in compInfo.find('div',class_='company-requisites').findAll('dl'):
        if ci.find('dt').text=='Дата регистрации':
            c.setDate(ci.find('dd').text)
        if ci.find('dt').text=='Уставный капитал':
            c.setCapital(ci.find('dd').text)
    return c

In [ ]:
mainUrl='https://www.rusprofile.ru' #Основной URL
urls=['/codes/89220/','/codes/429110/'] #Список добавочных URL-ов

In [ ]:
ids=[]
for url in urls: #Проходимся по списку
    j=1
    #Т.к. список комнаний распологается на нескольких страницах, проходимся по всем
    while(True):
        resp=requests.get(mainUrl+url+str(j)+'/')
        #Как только заканчиваются страницы, цикл заканчивается 
        if resp.status_code !=200:
            break
        pageInfo=BeautifulSoup(resp.content,'lxml')
        #Формируется список ID всех компаний, по которым потом находится подробная информация
        for pi in pageInfo.findAll('div',class_='company-item__title'):
            ids.append(pi.find('a').get('href'))
        j+=1

Один поток

In [ ]:
for id_ in ids:
    companies.append(mainParser(id_))

Много потоков

In [ ]:
def addTiList(a):
    companies.append(mainParser(a))

threads = []
for id_ in ids:
    thread = threading.Thread(target=addTiList, args=(id_,))
    threads.append(thread)
    thread.start()
for thread in threads:
    thread.join()

Запись в БД

In [ ]:
con = pymysql.connect('localhost','root','root','rusprofile')
cursor = con.cursor()

In [ ]:
createTable='CREATE TABLE companies ('+\
    'id INT NOT NULL,'+\
    'name VARCHAR(100),'+\
    'ogrn VARCHAR(100),'+\
    'okpo VARCHAR(100),'+\
    'status VARCHAR(100),'+\
    'regDate TIMESTAMP(6),'+\
    'capital INT,'+\
    'PRIMARY KEY(id))'

In [ ]:
cursor.execute(createTable)

In [ ]:
insertSQL='INSERT INTO companies (id, name, ogrn, okpo, status, regDate, capital) VALUES (%s, %s, %s, %s, %s, %s, %s)'

In [ ]:
for i in companies:
    cursor.execut(insertSQL,(i.id_,i.name,i.ogrn,i.okpo,i.status,i.regDate,i.capital))
    con.commit()